# Notes:
* Print out the coef for the Linear and Logistic model after using the Recursive Feature Selection
* Do import Dependencies in each section
* Stock indexes and Unemployment
* Write out equations for each model based on coef of each variable
* 


Market Cap (Xmc)
US Dollar Index (Xusdi)
Stock Avg. close (Xsac)
Unemployment % (Xup)
Y = predicted price? 
Eq: Y = .0019067 + 9.91451846e-01(Xmc) + -5.29526037e-03(Xusdi) + 3.79283974e-04(Xsac) + 4.57715256e-03(Xup)
actual price for today 10/26
Y= .0019067 + 9.91451846e-01(237040452187) + -5.29526037e-03(92.966) + 3.79283974e-04(14408) + 4.57715256e-03(Xup)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Import libraries from Sklearn for Machine Learning
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression ,Lasso
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE

In [2]:
# Read in the files
consolidated_file=pd.read_csv('Resources/consolidated_coin_data.csv')
income_data=pd.read_csv('Resources/cleaned_disposable_inc.csv')

In [3]:
# Convert to DataFrame
consolidated_df=pd.DataFrame(consolidated_file)
income_df=pd.DataFrame(income_data)

In [4]:
# Convert the data to float for last 6 columns
column_list=['Open','High','Low','Close','Volume','Market Cap']
for x in column_list:
    consolidated_df[x] = consolidated_df[x].str.replace(',', '').astype(float)

In [5]:
# Rename columns for consolidated_df
consolidated_df=consolidated_df.rename(columns={'Currency':'Name','Market Cap':'Market_Cap'})

In [6]:
# Double check data type for the consolidated_df, crypto_df and income_df
print('Consolidated_df data types: ' ,consolidated_df.dtypes)
print('Income_df data types: ' ,income_df.dtypes)

Consolidated_df data types:  Name           object
Date           object
Open          float64
High          float64
Low           float64
Close         float64
Volume        float64
Market_Cap    float64
dtype: object
Income_df data types:  Unnamed: 0      int64
Date           object
Income        float64
Month           int64
Month_Year     object
dtype: object


In [7]:
# Display the consolidated_df
consolidated_df

,Name,Date,Open,High,Low,Close,Volume,Market_Cap
0,tezos,"Dec 04, 2019",1.29,1.32,1.25,1.25,46048752.0,824588509.0
1,tezos,"Dec 03, 2019",1.24,1.32,1.21,1.29,41462224.0,853213342.0
2,tezos,"Dec 02, 2019",1.25,1.26,1.20,1.24,27574097.0,817872179.0
3,tezos,"Dec 01, 2019",1.33,1.34,1.25,1.25,24127567.0,828296390.0
4,tezos,"Nov 30, 2019",1.31,1.37,1.31,1.33,28706667.0,879181680.0
...,...,...,...,...,...,...,...,...
28939,bitcoin-sv,"May 02, 2013",3.78,4.04,3.01,3.37,0.0,58287979.0
28940,bitcoin-sv,"May 01, 2013",4.29,4.36,3.52,3.80,0.0,65604596.0
28941,bitcoin-sv,"Apr 30, 2013",4.40,4.57,4.17,4.30,0.0,74020918.0
28942,bitcoin-sv,"Apr 29, 2013",4.37,4.57,4.23,4.38,0.0,75388964.0


In [8]:
# Set the index of consolidated_df to Name, in order to filter out the bitcoin rows
consolidated_df=consolidated_df.set_index('Name')

In [9]:
# Filter out for just bitcoin
btc_df=consolidated_df.loc['bitcoin']

In [10]:
# Display the btc_df
btc_df

,Date,Open,High,Low,Close,Volume,Market_Cap
Name,,,,,,,
bitcoin,"Dec 04, 2019",7320.13,7539.78,7170.92,7252.03,2.166424e+10,1.311431e+11
bitcoin,"Dec 03, 2019",7323.98,7418.86,7229.36,7320.15,1.479749e+10,1.323599e+11
bitcoin,"Dec 02, 2019",7424.04,7474.82,7233.40,7321.99,1.708204e+10,1.323790e+11
bitcoin,"Dec 01, 2019",7571.62,7571.62,7291.34,7424.29,1.872071e+10,1.342151e+11
bitcoin,"Nov 30, 2019",7764.06,7836.10,7515.85,7569.63,1.715819e+10,1.368294e+11
...,...,...,...,...,...,...,...
bitcoin,"May 02, 2013",116.38,125.60,92.28,105.21,0.000000e+00,1.168517e+09
bitcoin,"May 01, 2013",139.00,139.89,107.72,116.99,0.000000e+00,1.298955e+09
bitcoin,"Apr 30, 2013",144.00,146.93,134.05,139.00,0.000000e+00,1.542813e+09


In [11]:
# Add the Y_Average for the new_df
btc_df['BTC_Average']=(btc_df['Open']+btc_df['High']+btc_df['Low']+btc_df['Close'])/4

C:\Users\phuph\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# Convert the Date column of btc_df
btc_df['Date']=pd.to_datetime(btc_df['Date'])

C:\Users\phuph\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
btc_df

,Date,Open,High,Low,Close,Volume,Market_Cap,BTC_Average
Name,,,,,,,,
bitcoin,2019-12-04,7320.13,7539.78,7170.92,7252.03,2.166424e+10,1.311431e+11,7320.7150
bitcoin,2019-12-03,7323.98,7418.86,7229.36,7320.15,1.479749e+10,1.323599e+11,7323.0875
bitcoin,2019-12-02,7424.04,7474.82,7233.40,7321.99,1.708204e+10,1.323790e+11,7363.5625
bitcoin,2019-12-01,7571.62,7571.62,7291.34,7424.29,1.872071e+10,1.342151e+11,7464.7175
bitcoin,2019-11-30,7764.06,7836.10,7515.85,7569.63,1.715819e+10,1.368294e+11,7671.4100
...,...,...,...,...,...,...,...,...
bitcoin,2013-05-02,116.38,125.60,92.28,105.21,0.000000e+00,1.168517e+09,109.8675
bitcoin,2013-05-01,139.00,139.89,107.72,116.99,0.000000e+00,1.298955e+09,125.9000
bitcoin,2013-04-30,144.00,146.93,134.05,139.00,0.000000e+00,1.542813e+09,140.9950


In [14]:
# Drop out Unnamed column in income_df
income_df=income_df.drop(columns={'Unnamed: 0'},axis=1)

In [18]:
# Convert the Date column in income_df
income_df['Date']=pd.to_datetime(income_df['Date'])

In [19]:
# Join the new_df with the income_df
income_btc_df=income_df.merge(btc_df,on='Date')

In [20]:
# Display the income_btc_df
income_btc_df

,Date,Income,Month,Month_Year,Open,High,Low,Close,Volume,Market_Cap,BTC_Average
0,2013-05-01,39054.0,5,2013-05,139.00,139.89,107.72,116.99,0.0,1.298955e+09,125.9000
1,2013-06-01,39071.0,6,2013-06,128.82,129.78,127.20,129.30,0.0,1.451916e+09,128.7750
2,2013-07-01,39004.0,7,2013-07,97.51,97.66,86.30,88.05,0.0,1.000065e+09,92.3800
3,2013-08-01,39063.0,8,2013-08,106.21,108.00,103.01,104.00,0.0,1.195225e+09,105.3050
4,2013-09-01,39194.0,9,2013-09,135.14,145.81,134.81,138.34,0.0,1.610215e+09,138.5250
5,2013-10-01,39064.0,10,2013-10,132.68,134.63,131.55,132.18,0.0,1.557001e+09,132.7600
6,2013-11-01,39165.0,11,2013-11,203.90,206.65,202.13,206.18,0.0,2.460743e+09,204.7150
7,2013-12-01,39265.0,12,2013-12,1128.92,1133.08,801.82,955.85,0.0,1.153171e+10,1004.9175
8,2014-01-01,39432.0,1,2014-01,754.97,775.35,754.97,771.40,22489400.0,9.403308e+09,764.1725
9,2014-02-01,39655.0,2,2014-02,828.61,853.52,827.11,832.58,19668700.0,1.027520e+10,835.4550


In [ ]:
# Drop some columns
income_btc_df=income_btc_df[['Date','Income','Open','High','Low','Close','Volume','Market_Cap','BTC_Average']]

In [ ]:
# Select the X and y for the machine
X=income_btc_df.drop(['Date','High','Open','Close','Low','BTC_Average'],axis=1)
y=income_btc_df['BTC_Average'].values.reshape(-1, 1)

In [ ]:
print(X.shape,y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
# Create a Linear Regression model
model = LinearRegression()

In [ ]:
# Fit in the model
model.fit(X_train_scaled,y_train_scaled)

In [ ]:
print(model)

In [ ]:
print('Weight coefficients: ', model.coef_)
print('y-axis intercept: ', model.intercept_) 

In [ ]:
predictions = model.predict(X_test_scaled)
model.fit(X_train_scaled, y_train_scaled)
plt.scatter(model.predict(X_train_scaled), model.predict(X_train_scaled) - y_train_scaled, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_scaled), model.predict(X_test_scaled) - y_test_scaled, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test_scaled.min(), xmax=y_test_scaled.max())
plt.title("Residual Plot")
plt.show()


In [ ]:
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual stats: {y_test_scaled[:10]}")

In [ ]:
y_new = y_scaler.inverse_transform(y_test_scaled)


In [ ]:
y_new1 = y_scaler.inverse_transform(predictions)


In [ ]:
# Actual values
y_new

In [ ]:
# PRediction values 
y_new1

In [ ]:
# Read in unemployment and USD index
unemployment_USD_index=pd.read_csv('Resources/unemployment_and_USDI_data.csv')

In [ ]:
unemployment_USDI_df=pd.DataFrame(unemployment_USD_index)

In [ ]:
# Merge with the crypto_df
unemployment_vs_btc=crypto_df.merge(unemployment_USDI_df,on='Date',how='left')

In [ ]:
crypto_df=crypto_df.reset_index()

In [ ]:
# Merge with the crypto_df
unemployment_vs_btc=crypto_df.merge(unemployment_USDI_df,on='Date',how='left')

In [ ]:
unemployment_vs_btc

In [ ]:
unemployment_vs_btc=unemployment_vs_btc.set_index('Name')

In [ ]:
final_btc_df=unemployment_vs_btc.loc['bitcoin']

In [ ]:
final_btc_df

In [ ]:
final_btc_df['Y_Average']=(final_btc_df['Open']+final_btc_df['High']+final_btc_df['Low']+final_btc_df['Close'])/4

In [ ]:
final_btc_df=final_btc_df.rename(columns={'Unemployment Value ':'Unemployment_Percentage','US Dollar Index ':'USDI'})

In [ ]:
final_btc_df.tail(20)

In [ ]:
X=final_btc_df[['Volume','Market_Cap','Spread','Unemployment_Percentage','USDI']]
y=final_btc_df['Y_Average'].values.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25)


In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train_scaled,y_train_scaled)

In [ ]:
print('Weight coefficients: ', model.coef_)
print('y-axis intercept: ', model.intercept_) 

In [ ]:
predictions = model.predict(X_test_scaled)
model.fit(X_train_scaled, y_train_scaled)
plt.scatter(model.predict(X_train_scaled), model.predict(X_train_scaled) - y_train_scaled, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_scaled), model.predict(X_test_scaled) - y_test_scaled, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test_scaled.min(), xmax=y_test_scaled.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual stats: {y_test_scaled[:10]}")

# Finalize the DataFrame with stocks indexes, Unemployment and USDI


In [ ]:
dow_file=pd.read_csv('Resources/DOW.csv')
nasdaq_file=pd.read_csv('Resources/NASDAQ.csv')
sp500_file=pd.read_csv('Resources/SP500.csv')


In [ ]:
dow_df=pd.DataFrame(dow_file)
nasdaq_df=pd.DataFrame(nasdaq_file)
sp500_df=pd.DataFrame(sp500_file)

In [ ]:
dow_df['DOW_Average']=(dow_df['Open']+dow_df['High']+dow_df['Low']+dow_df['Close'])/4
nasdaq_df['NASDAQ_Average']=(nasdaq_df['Open']+nasdaq_df['High']+nasdaq_df['Low']+nasdaq_df['Close'])/4
sp500_df['SP500_Average']=(sp500_df['Open']+sp500_df['High']+sp500_df['Low']+sp500_df['Close'])/4


In [ ]:
dow_df=dow_df.rename(columns={'Adj Close':'Adjusted_Close'})
nasdaq_df=nasdaq_df.rename(columns={'Adj Close':'Adjusted_Close'})
sp500_df=sp500_df.rename(columns={'Adj Close':'Adjusted_Close'})

In [ ]:
dow_df=dow_df[['Date','Adjusted_Close']]
nasdaq_df=nasdaq_df[['Date','Adjusted_Close']]
sp500_df=sp500_df[['Date','Adjusted_Close']]

In [ ]:
dow_df['Date']=pd.to_datetime(dow_df['Date'])

In [ ]:
nasdaq_df['Date']=pd.to_datetime(nasdaq_df['Date'])
sp500_df['Date']=pd.to_datetime(sp500_df['Date'])

In [ ]:
final_btc_df['Date']=final_btc_df['Date'].astype('datetime64[ns]')

In [ ]:
final_btc_df=final_btc_df.merge(dow_df,on='Date',how='left')

In [ ]:
final_btc_df=final_btc_df.merge(nasdaq_df,on='Date',how='left')

In [ ]:
final_btc_df=final_btc_df.merge(sp500_df,on='Date',how='left')

In [ ]:
final_btc_df=final_btc_df.rename(columns={'Adjusted_Close_x':'DOW_Close','Adjusted_Close_y':'NASDAQ_Close','Adjusted_Close':'SP500_Close'})

In [ ]:
final_btc_df

In [ ]:
all_data_df=final_btc_df[~pd.isnull(final_btc_df['DOW_Close'])]

In [ ]:
all_data_df

In [ ]:
X=all_data_df[['Volume','Market_Cap','Spread','Unemployment_Percentage','USDI','DOW_Close','NASDAQ_Close','SP500_Close']]
y=all_data_df['Y_Average'].values.reshape(-1,1)

In [ ]:
print(X.shape,y.shape)

In [ ]:
model=LinearRegression()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.6, random_state=25)


In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
model.fit(X_train_scaled,y_train_scaled)

In [ ]:
print('Weight coefficients: ', model.coef_)
print('y-axis intercept: ', model.intercept_) 

In [ ]:
predictions = model.predict(X_test_scaled)
model.fit(X_train_scaled, y_train_scaled)
plt.scatter(model.predict(X_train_scaled), model.predict(X_train_scaled) - y_train_scaled, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_scaled), model.predict(X_test_scaled) - y_test_scaled, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test_scaled.min(), xmax=y_test_scaled.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual stats: {y_test_scaled[:10]}")

In [ ]:
y_new = y_scaler.inverse_transform(y_test_scaled)
y_new1 = y_scaler.inverse_transform(predictions)


In [ ]:
len(y_new)

In [ ]:
y_new1

In [ ]:
all_data_df['Stock_Average_Close']=(all_data_df['DOW_Close']+all_data_df['NASDAQ_Close']+all_data_df['SP500_Close'])/3

In [ ]:
X=all_data_df[['Volume','Market_Cap','Spread','Unemployment_Percentage','USDI','DOW_Close','NASDAQ_Close','SP500_Close']]
y=all_data_df['Y_Average']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.8, random_state=25)


In [ ]:
clf = Lasso(max_iter=20000)
parameters={'alpha':[1e-15,1e-10,1e-8]}
lasso_regressor=GridSearchCV(clf,parameters,scoring='neg_mean_squared_error',cv=10)
lasso_regressor.fit(X_train,y_train)
clf.fit(X_train,y_train)

In [ ]:
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

In [ ]:
predictions = clf.predict(X_test)
clf.fit(X_train, y_train)
plt.scatter(clf.predict(X_train), clf.predict(X_train) - y_train, c="blue", label="Training Data")
plt.scatter(clf.predict(X_test), clf.predict(X_test) - y_test, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual stats: {y_test[:10]}")

In [ ]:
from sklearn.feature_selection import VarianceThreshold


In [ ]:
X=all_data_df[['Volume','Market_Cap','Spread','Unemployment_Percentage','USDI','DOW_Close','NASDAQ_Close','SP500_Close']]
y=all_data_df['Y_Average']

In [ ]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

In [ ]:
sel.fit_transform(X)

# Drop out the Spread column and Append the Up/Down Column

In [ ]:
all_data_df=all_data_df.drop(columns={'Spread'},axis=1)

In [ ]:
all_data_df

In [ ]:
X=all_data_df[['Volume','Market_Cap','Unemployment_Percentage','USDI','Stock_Average_Close']]
y=all_data_df['Y_Average'].values.reshape(-1,1)

In [ ]:
model=LinearRegression()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.8, random_state=25)


In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
model.fit(X_train_scaled,y_train_scaled)

In [ ]:
predictions = model.predict(X_test_scaled)
model.fit(X_train_scaled, y_train_scaled)
plt.scatter(model.predict(X_train_scaled), model.predict(X_train_scaled) - y_train_scaled, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_scaled), model.predict(X_test_scaled) - y_test_scaled, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test_scaled.min(), xmax=y_test_scaled.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
print('Weight coefficients: ', model.coef_)
print('y-axis intercept: ', model.intercept_) 

In [ ]:
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual stats: {y_test_scaled[:10]}")

In [ ]:
y_actual = y_scaler.inverse_transform(y_test_scaled)
y_predicted = y_scaler.inverse_transform(predictions)

In [ ]:
y_actual

In [ ]:
y_predicted

In [ ]:
up_down_list=list(all_data_df['Close']-all_data_df['Open'])

up_down_list


In [ ]:
up_down_list = ['Up' if x >0 else 'Down' for x in up_down_list]
up_down_list

In [ ]:
all_data_df['Movement']=up_down_list

In [ ]:
all_data_df

In [ ]:
label_endoded=pd.get_dummies(all_data_df['Movement'])

In [ ]:
label_endoded

In [ ]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 
all_data_df['Label_Encoded']= label_encoder.fit_transform(all_data_df['Movement']) 


In [ ]:
all_data_df

In [ ]:
X=all_data_df[['Volume','Market_Cap','Unemployment_Percentage','USDI','Stock_Average_Close','Y_Average']]
y=all_data_df['Label_Encoded']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
print(X_train.shape,y_train.shape)

In [ ]:
model = LogisticRegression(solver='liblinear',C=10, random_state=42)


In [ ]:
model.fit(X_train,y_train)

In [ ]:
print(model.classes_)
print(model.intercept_)
print(model.coef_)


In [ ]:
print(model.predict_proba(X_train))


In [ ]:
model.predict(X_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
print(classification_report(y_train, model.predict(X_train)))

In [ ]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)


In [ ]:
X=all_data_df[['Volume','Market_Cap','Unemployment_Percentage','USDI','Stock_Average_Close']]
y = all_data_df.loc[:, all_data_df.columns == 'Label_Encoded']


In [ ]:
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['Label_Encoded'])

In [ ]:
data_final_vars=all_data_df.columns.values.tolist()

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary())

In [ ]:
X=all_data_df[['Volume','Market_Cap','USDI','Stock_Average_Close']]
y = all_data_df.loc[:, all_data_df.columns == 'Label_Encoded']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
model = LogisticRegression(solver='liblinear',C=10, random_state=42)
model.fit(X_train,y_train)
model.predict(X_train)
model.score(X_train, y_train)
print(classification_report(y_train, model.predict(X_train)))

In [ ]:
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['Label_Encoded'])

In [ ]:
data_final_vars=all_data_df.columns.values.tolist()

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
# to test out the coef of the LogisticRegression
print(model.coef_)
print(model.intercept_)

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary())

In [ ]:
X=all_data_df[['Volume','Market_Cap','USDI','Stock_Average_Close','Unemployment_Percentage']]
y = all_data_df['Y_Average'].values.reshape(-1,1)


In [ ]:
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 3)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
nof_list=np.arange(1,5)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))


In [ ]:
cols = list(X.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 4)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

In [ ]:
X=all_data_df[['Market_Cap', 'USDI', 'Stock_Average_Close', 'Unemployment_Percentage']]
y=all_data_df['Y_Average'].values.reshape(-1,1)

In [ ]:
model=LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.5, random_state=0)
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
model.fit(X_train_scaled,y_train_scaled)
predictions = model.predict(X_test_scaled)
plt.scatter(model.predict(X_train_scaled), model.predict(X_train_scaled) - y_train_scaled, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_scaled), model.predict(X_test_scaled) - y_test_scaled, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test_scaled.min(), xmax=y_test_scaled.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
print('Weight coefficients: ', model.coef_)b
print('y-axis intercept: ', model.intercept_) 
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual stats: {y_test_scaled[:10]}")

In [ ]:
coef_dict = {}
for coef, feat in zip(model.coef_[0,:],X):
    coef_dict[feat] = coef

In [ ]:
print(coef_dict)

In [ ]:
print(0.9914518460877908
-0.005295260369616241+
0.00037928397432413006+
0.004577152558298293)

In [ ]:
y_actual = y_scaler.inverse_transform(y_test_scaled)
y_predicted = y_scaler.inverse_transform(predictions)

In [ ]:
y_actual

In [ ]:
y_predicted

In [ ]:
test_df=all_data_df

In [ ]:
test_df=test_df.set_index('Y_Average')

In [ ]:
print(y_actual[1])

In [ ]:
test_df.loc[236.515]